In [2]:
#import dependenceis
import pandas as pd
import sqlalchemy
from sqlalchemy import inspect, create_engine
import numpy as np
import psycopg2

In [3]:
# bring in countries by continent from url

url = "https://statisticstimes.com/geography/countries-by-continents.php"

countries = pd.read_html(url)
countries_df = countries[2].copy()
countries_df.fillna("")

,No,Country or Area,ISO-alpha3 Code,M49 Code,Region 1,Region 2,Continent
0,1,Afghanistan,AFG,4,Southern Asia,,Asia
1,2,Åland Islands,ALA,248,Northern Europe,,Europe
2,3,Albania,ALB,8,Southern Europe,,Europe
3,4,Algeria,DZA,12,Northern Africa,,Africa
4,5,American Samoa,ASM,16,Polynesia,,Oceania
...,...,...,...,...,...,...,...
244,245,Wallis and Futuna Islands,WLF,876,Polynesia,,Oceania
245,246,Western Sahara,ESH,732,Northern Africa,,Africa
246,247,Yemen,YEM,887,Western Asia,,Asia
247,248,Zambia,ZMB,894,Eastern Africa,Sub-Saharan Africa,Africa


### Import Crops

In [7]:
crops = pd.read_csv("../project_2_whats_for_breakfast/data/crop1.csv")
crops_df = crops
crops_df.head()

,Area,Item,Element,Year,Unit,Value
0,Afghanistan,"Almonds, with shell",Area harvested,1975,ha,0.0
1,Afghanistan,"Almonds, with shell",Area harvested,1976,ha,5900.0
2,Afghanistan,"Almonds, with shell",Area harvested,1977,ha,6000.0
3,Afghanistan,"Almonds, with shell",Area harvested,1978,ha,6000.0
4,Afghanistan,"Almonds, with shell",Area harvested,1979,ha,6000.0


### Import Livestock

In [8]:
livestock = pd.read_csv("../project_2_whats_for_breakfast/data/live1.csv")
livestock_df = livestock
livestock_df.head()

,Area,Item,Element,Year,Unit,Value
0,Afghanistan,Asses,Stocks,1961,Head,1300000.0
1,Afghanistan,Asses,Stocks,1962,Head,851850.0
2,Afghanistan,Asses,Stocks,1963,Head,1001112.0
3,Afghanistan,Asses,Stocks,1964,Head,1150000.0
4,Afghanistan,Asses,Stocks,1965,Head,1300000.0


### Start Transformation

In [9]:
# Trim crops
trim_crops = crops.loc[(crops['Year']>1979)]
#trim_crops.to_csv("../project_2_whats_for_breakfast/data/trimmed_crops.csv")
trimmed_crops_df = trim_crops.copy()
trimmed_crops_df.head()


,Area,Item,Element,Year,Unit,Value
5,Afghanistan,"Almonds, with shell",Area harvested,1980,ha,5800.0
6,Afghanistan,"Almonds, with shell",Area harvested,1981,ha,5800.0
7,Afghanistan,"Almonds, with shell",Area harvested,1982,ha,5800.0
8,Afghanistan,"Almonds, with shell",Area harvested,1983,ha,5700.0
9,Afghanistan,"Almonds, with shell",Area harvested,1984,ha,5700.0


In [10]:
# Trim Livestock
trim_livestock = livestock.loc[(livestock['Year']>1979)]
#trim_livestock.to_csv("../project_2_whats_for_breakfast/data/trimmed_livestock.csv")
trimmed_livestock_df = trim_livestock.copy()
trimmed_crops_df.head()

,Area,Item,Element,Year,Unit,Value
5,Afghanistan,"Almonds, with shell",Area harvested,1980,ha,5800.0
6,Afghanistan,"Almonds, with shell",Area harvested,1981,ha,5800.0
7,Afghanistan,"Almonds, with shell",Area harvested,1982,ha,5800.0
8,Afghanistan,"Almonds, with shell",Area harvested,1983,ha,5700.0
9,Afghanistan,"Almonds, with shell",Area harvested,1984,ha,5700.0


In [11]:
# Create Crop Items DF
crop_items = trimmed_crops_df[['Item', 'Element']].copy()
crop_items_dim = pd.DataFrame(crop_items, columns=['Item', 'Element'])
crop_items_dim = crop_items_dim.drop_duplicates()
crop_items_dim


,Item,Element
5,"Almonds, with shell",Area harvested
50,"Almonds, with shell",Yield
96,"Almonds, with shell",Production
156,"Anise, badian, fennel, coriander",Area harvested
197,"Anise, badian, fennel, coriander",Yield
...,...,...
231707,Cloves,Production
242807,Tallowtree seed,Area harvested
242848,Tallowtree seed,Yield
242897,Tallowtree seed,Production


In [12]:
# Creat Livestock Items DF
livestock_items = trimmed_livestock_df[['Item', 'Element']].copy()
livestock_items_dim = pd.DataFrame(livestock_items, columns=['Item', 'Element'])
livestock_items_dim = livestock_items_dim.drop_duplicates()
livestock_items_dim

,Item,Element
19,Asses,Stocks
79,Camels,Stocks
139,Cattle,Stocks
199,Chickens,Stocks
259,Goats,Stocks
319,Horses,Stocks
379,Mules,Stocks
439,Sheep,Stocks
559,Beehives,Stocks
619,Buffaloes,Stocks


### Append Crop & Livestock items

In [13]:
items_dimension = crop_items_dim.append(livestock_items_dim, ignore_index=True)
items_dimension

/var/folders/49/s36vhyf936d19tpm9b2yqny00000gn/T/ipykernel_2649/3392248888.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  items_dimension = crop_items_dim.append(livestock_items_dim, ignore_index=True)


,Item,Element
0,"Almonds, with shell",Area harvested
1,"Almonds, with shell",Yield
2,"Almonds, with shell",Production
3,"Anise, badian, fennel, coriander",Area harvested
4,"Anise, badian, fennel, coriander",Yield
...,...,...
363,Buffaloes,Stocks
364,Ducks,Stocks
365,Geese and guinea fowls,Stocks
366,Pigs,Stocks


### Create year dimension DF

In [14]:
year = trim_crops['Year']
year = year.drop_duplicates()
year = pd.DataFrame(year, columns=['Year'])
year.head()

,Year
5,1980
6,1981
7,1982
8,1983
9,1984


### Create Database Connection

In [15]:
protocol = 'postgresql'
username = 'postgres'
password = 'bootcamp'
host = 'localhost'
port = 5432
db_name = 'captain_crunch'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{db_name}'
engine = create_engine(rds_connection_string)


### Inspect Tables

In [16]:
inspector = inspect(engine)
inspector.get_table_names()

['tbl_countries',
 'tbl_crops',
 'tbl_item',
 'tbl_year',
 'tbl_livestock',
 'tbl_population']